In [1]:
import numpy as np
import matplotlib.pyplot as plt 
from skimage.morphology import skeletonize
import cv2
import json
from sklearn.svm import SVC
from skimage.morphology import skeletonize
import cv2

####################################################################
####left data
####################################################################
file_name = './2nd_process_left_data.csv' # 좌측 전처리 완료 데이터 선택

left_vectors = np.zeros((414,80)) # 전체데이터를 저장할 변수 선언

r = open(file_name, mode= 'r') # 데이터 파일 open 
l_lines = r.readlines() # 파일에서 변수로 값 불러오기
l_lines[0]

for data in range(0,414):

        numbers = l_lines[data].split(',') #각 행에서 comma 로 데이터를 구분하여 변수에 저장 

        flir_vector = [] # 행 데이터를 저장할 변수 선언 
        for i in range(0,80):
            flir_vector.append(float(numbers[i])) # 데이터를 순서대로 붙여 행에 저장
        
        left_vectors[data,:] = np.transpose(flir_vector) # 전체 데이터 변수에 저장

# plt.figure(1)
# plt.plot(np.arange(0,80),left_vectors[400]) #400번째 전처리 제품 시각화
# plt.show()


# 전처리 완료 라벨 가져오기
import json

with open('2nd_process_left_label.json', 'r') as infile:
           newlist_left = json.load(infile)
        
print(newlist_left)

# 전처리 완료 라벨 append 전 dimension 맞추기
left_label = np.zeros((len(newlist_left),1))

num = 0

for val in newlist_left:
    left_label[num] = newlist_left[num]
    num = num + 1


####################################################################
####right data
####################################################################
file_name = './2nd_process_right_data.csv' # 좌측 전처리 완료 데이터 선택

right_vectors = np.zeros((423,80)) # 전체데이터를 저장할 변수 선언

r = open(file_name, mode= 'r') # 데이터 파일 open 
r_lines = r.readlines() # 파일에서 변수로 값 불러오기
r_lines[0]

for data in range(0,423):

        numbers = r_lines[data].split(',') #각 행에서 comma 로 데이터를 구분하여 변수에 저장 

        flir_vector = [] # 행 데이터를 저장할 변수 선언 
        for i in range(0,80):
            flir_vector.append(float(numbers[i])) # 데이터를 순서대로 붙여 행에 저장
        
        right_vectors[data,:] = np.transpose(flir_vector) # 전체 데이터 변수에 저장

# plt.figure(1)
# plt.plot(np.arange(0,80),left_vectors[400]) #400번째 전처리 제품 시각화
# plt.show()


# 전처리 완료 라벨 가져오기
import json

with open('2nd_process_right_label.json', 'r') as infile:
           newlist_right = json.load(infile)
        
print(newlist_right)

# 전처리 완료 라벨 append 전 dimension 맞추기
right_label = np.zeros((len(newlist_right),1))

num = 0

for val in newlist_right:
    right_label[num] = newlist_right[num]
    num = num + 1

#################################################################
####학습하고 예측모델 만들어 예측하기
#################################################################

import warnings # 경고문을 꺼주는 코드입니다

warnings.filterwarnings(action='ignore') 


# SVM 훈련 코드입니다.

from sklearn.svm import SVC
import joblib

# 좌측 데이터와 좌측 라벨링 append => (414x81)
input_left_datas = np.hstack((left_vectors, left_label))
input_right_datas = np.hstack((right_vectors, right_label))

input_total_datas = np.vstack((input_right_datas, input_left_datas))    ##우측, 좌측 세로로 붙임

wrong_guess_stack= 0
wrong_guess_buffer = np.zeros((1000,1))

for i in range(0,1000):
    #np.random.shuffle(input_left_datas) # 데이터를 랜덤 셔플
    #np.random.shuffle(input_right_datas) # 데이터를 랜덤 셔플
    np.random.shuffle(input_total_datas)
    # lsX = input_left_datas[0:414,0:80]      #훈련용 데이터
    # lsXt = input_left_datas[0:414,0:80]  # 테스트용 데이터
    sX = input_left_datas[0:837,0:80]      #훈련용 데이터
    sXt = input_left_datas[0:837,0:80]  # 테스트용 데이터
    # lsY = input_left_datas[0:414,80:81] #훈련용 라벨
    # lsYt = input_left_datas[0:414,80:81] #테스트용 라벨
    sY = input_left_datas[0:837,80:81] #훈련용 라벨
    sYt = input_left_datas[0:837,80:81] #테스트용 라벨
    # rsX = input_right_datas[0:423,0:80]
    # rsXt = input_right_datas[0:423,0:80]  

    # rsY = input_right_datas[0:423,80:81]
    # rsYt = input_right_datas[0:423,80:81]

    model = SVC(kernel='rbf') # 비선형 SVM kernel 모델 설정
    # model.fit(lsX,lsY) # SVM model 훈련
    # model.fit(rsX,rsY) # SVM model 훈련
    model.fit(sX,sY)

    # left_result = model.predict(lsXt)
    # right_result = model.predict(lsYt)
    total_result = model.predict(sXt)


    wrong_guess = (np.sum(np.abs(total_result-np.transpose(sYt)))) # 틀린 예측 개수
    wrong_guess_buffer[i, 0] = wrong_guess
    wrong_guess_stack = wrong_guess_stack + wrong_guess

print(wrong_guess)
print(wrong_guess_stack)
print('오류율 : ', (wrong_guess_stack/1000/837)*100 ) # 100

joblib.dump(model, './test_total_model123.pkl')



[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 1.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 1.0, 1.0, 1.0, 0.0, 0.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0,

['./test_total_model123.pkl']

In [2]:
test_model = joblib.load('./test_total_model123.pkl')

wrong_guess_stack= 0
wrong_guess = 0
predict_result = np.zeros((100,1))
for i in range(0,100):

    np.random.shuffle(input_total_datas) # 데이터를 랜덤 셔플
 
    sXt = input_total_datas[0:100,0:80] # 테스트용 데이터

    sYt = input_total_datas[0:100,80:81] # 테스트용 라벨

    s_result = test_model.predict(sXt)

    wrong_guess = (np.sum(np.abs(s_result-np.transpose(sYt)))) # 틀린 예측 개수
    wrong_guess_stack = wrong_guess_stack + wrong_guess
    
print(wrong_guess)
print(wrong_guess_stack)
print('오류율 : ', (wrong_guess_stack/100/100)*100) # 100

13.0
1119.0
오류율 :  11.19


In [8]:
test_model = joblib.load('./test_total_model.pkl')

wrong_guess_stack= 0
wrong_guess = 0

for i in range(0,200):

    np.random.shuffle(input_total_datas) # 데이터를 랜덤 셔플
 
    sXt = input_total_datas[0:837,0:80] # 테스트용 데이터

    sYt = input_total_datas[0:837,80:81] # 테스트용 라벨

    s_result = test_model.predict(sXt)

    wrong_guess = (np.sum(np.abs(s_result-np.transpose(sYt)))) # 틀린 예측 개수
    wrong_guess_stack = wrong_guess_stack + wrong_guess
    
print(wrong_guess)
print(wrong_guess_stack)
print('오류율 : ', (wrong_guess_stack/200/837)*100) # 100

90.0
18000.0
오류율 :  10.75268817204301
